In [1]:
from PIL import Image
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# convert image into numpy array

In [2]:
data = np.empty((22424,480,640,3), dtype = np.uint8 )

label = np.empty(22424, dtype = np.uint8)

train_path = '/home/minheng/train'
test_path = '/home/minheng/test'

def load_data(path):
    i = 0
    for name in os.listdir(path):
        rank = int(name[1])
        file_path = os.path.join(path,name)
        for pic in os.listdir(file_path):
            pic_name = os.path.join(file_path, pic)
            data[i] = np.array(Image.open(pic_name),dtype = np.uint8)
            label[i] = rank
            i += 1
            
load_data(train_path)

# use 1000 samples for base case testing

In [3]:
slicer = np.random.randint(0, high = 22424, size = 1000)

x = data[slicer]

x = x.reshape(x.shape[0], -1)

y = label[slicer]

# train_test split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# fit model (base model : KNN)

In [10]:
neighbor = KNeighborsClassifier(n_neighbors=5)
neighbor.fit(X_train, y_train)
neighbor.score(X_test,y_test)

0.65500000000000003